# Imports

In [24]:
import pybaseball as pyb
import pandas as pd
import time  # Import the time module
import inspect

# Functions

In [9]:
pyb.cache.enable()
pyb.cache.config.cache_type='csv'
pyb.cache.config.save()

In [10]:

# get the register data
chad = pyb.chadwick_register()

# get the register data and save to disk
chad = pyb.chadwick_register(save=True)

In [11]:
# import pkg_resources

# def get_version(package_name):
#     try:
#         return pkg_resources.get_distribution(package_name).version
#     except pkg_resources.DistributionNotFound:
#         return "Package not found"

# print(get_version("pybaseball"))

In [21]:


def bref_batter(date1,date2):
# Initialize a list to store all data
    all_data = []

# Loop over the years from the beginning year to the current year
    for year in range(date1, date2):  # Replace 2024 with the current year
        try:
            data = pyb.batting_stats_bref(year)
            print(f"Data fetched for year: {year}")
            all_data.append(data)
        except Exception as e:
            print(f"An error occurred for year: {year}. Error: {str(e)}")
    

    time.sleep(4)
# Concatenate all dataframes in the list
    all_data_df = pd.concat(all_data)
    return all_data_df

date1=2008
date2=2024
batter_data=bref_batter(date1,date2)

Data fetched for year: 2008
Data fetched for year: 2009
Data fetched for year: 2010
Data fetched for year: 2011
Data fetched for year: 2012
Data fetched for year: 2013
Data fetched for year: 2014
Data fetched for year: 2015
Data fetched for year: 2016
Data fetched for year: 2017
Data fetched for year: 2018
Data fetched for year: 2019
Data fetched for year: 2020
Data fetched for year: 2021
Data fetched for year: 2022
Data fetched for year: 2023


In [13]:
def bref_pitcher(date1,date2):
# Initialize a list to store all data
    all_data = []

# Loop over the years from the beginning year to the current year
    for year in range(date1, date2):  # Replace 2024 with the current year
        try:
            data = pyb.pitching_stats_bref(year)
            print(f"Data fetched for year: {year}")
            all_data.append(data)
        except Exception as e:
            print(f"An error occurred for year: {year}. Error: {str(e)}")
    

    time.sleep(4)
# Concatenate all dataframes in the list
    all_data_df = pd.concat(all_data)
    return all_data_df

date1=2008
date2=2024
pitcher_data=bref_pitcher(date1,date2)

Data fetched for year: 2008
Data fetched for year: 2009
Data fetched for year: 2010
Data fetched for year: 2011
Data fetched for year: 2012
Data fetched for year: 2013
Data fetched for year: 2014
Data fetched for year: 2015
Data fetched for year: 2016
Data fetched for year: 2017
Data fetched for year: 2018
Data fetched for year: 2019
Data fetched for year: 2020
Data fetched for year: 2021
Data fetched for year: 2022
Data fetched for year: 2023


In [34]:
print(inspect.getsource(pyb.pitching_stats_range))

@cache.df_cache()
def pitching_stats_range(start_dt: Optional[str]=None, end_dt: Optional[str]=None) -> pd.DataFrame:
    """
    Get all pitching stats for a set time range. This can be the past week, the
    month of August, anything. Just supply the start and end date in YYYY-MM-DD
    format.
    """
    # ensure valid date strings, perform necessary processing for query
    start_dt_date, end_dt_date = sanitize_date_range(start_dt, end_dt)
    if start_dt_date.year < 2008:
        raise ValueError("Year must be 2008 or later")
    if end_dt_date.year < 2008:
        raise ValueError("Year must be 2008 or later")
    # retrieve html from baseball reference
    soup = get_soup(start_dt_date, end_dt_date)
    table = get_table(soup)
    table = table.dropna(how='all') # drop if all columns are NA
    #fix some strange formatting for percentage columns
    table = table.replace('---%', np.nan)
    #make sure these are all numeric
    for column in ['Age', '#days', 'G', 'GS', 'W', 'L',

In [17]:

def bref_fielder(date1,date2):
# Initialize a list to store all data
    all_data = []

# Loop over the years from the beginning year to the current year
    for year in range(date1, date2):  # Replace 2024 with the current year
        try:
            data = pyb.statcast_outs_above_average(year, "all", 0)
            print(f"Data fetched for year: {year}")
            all_data.append(data)
        except Exception as e:
            print(f"An error occurred for year: {year}. Error: {str(e)}")
    

    time.sleep(4)
# Concatenate all dataframes in the list
    all_data_df = pd.concat(all_data)
    return all_data_df

date1=2008
date2=2024
fielder_data=bref_fielder(date1,date2)


Data fetched for year: 2008
Data fetched for year: 2009
Data fetched for year: 2010
Data fetched for year: 2011
Data fetched for year: 2012
Data fetched for year: 2013
Data fetched for year: 2014
Data fetched for year: 2015
Data fetched for year: 2016
Data fetched for year: 2017
Data fetched for year: 2018
Data fetched for year: 2019
Data fetched for year: 2020
Data fetched for year: 2021
Data fetched for year: 2022
Data fetched for year: 2023


In [18]:
print(fielder_data)
print(pitcher_data)
print(batter_data)

    last_name, first_name player_id display_team_name  year  \
0             Abreu, José    547989         White Sox  2016   
1          Ackley, Dustin    554429           Yankees  2016   
2       Adames, Cristhian    542436           Rockies  2016   
3             Adams, Matt    571431         Cardinals  2016   
4         Adrianza, Ehire    501303            Giants  2016   
..                    ...       ...               ...   ...   
553     Yelich, Christian    592885           Brewers  2023   
554           Yepez, Juan    660766         Cardinals  2023   
555     Yoshida, Masataka    807799           Red Sox  2023   
556          Young, Jacob    696285         Nationals  2023   
557          Young, Jared    676724              Cubs  2023   

    primary_pos_formatted fielding_runs_prevented outs_above_average  \
0                      1B                      -4                 -6   
1                      1B                       2                  3   
2                      SS  

In [ ]:
print(player_data)

<class 'pandas.core.frame.DataFrame'>
